In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import matplotlib.colors as mcolors
from matplotlib.cm import ScalarMappable
from matplotlib.colors import ListedColormap
import geopandas as gpd

In [ ]:
# read distribution data
distribution_data = pd.read_csv('//sce.eix.com/workgroup/SPE_DSA/Charles/IgnitionGoalSettingInput/Distribution_2015-2024.csv',low_memory=False)
# filter distribution data
distribution_data = distribution_data[(distribution_data['time']>='2017') & (distribution_data['time']<='2025')]
# create a date column
distribution_data['Date'] = pd.to_datetime(distribution_data['time']).dt.strftime('%Y-%m-%d')
# create a year column
distribution_data['Year'] = pd.to_datetime(distribution_data['time']).dt.year

In [ ]:
distribution_data

,time,id,max_max_fpi_2.0_tdd_fuels_index,mean_max_fpi_2.0_tdd_fuels_index,median_max_fpi_2.0_tdd_fuels_index,p75_max_fpi_2.0_tdd_fuels_index,p90_max_fpi_2.0_tdd_fuels_index,p95_max_fpi_2.0_tdd_fuels_index,max_mean_fpi_2.0_tdd_fuels_index,mean_mean_fpi_2.0_tdd_fuels_index,...,p90_p90_lfpw_using_dew_point_depression,p95_p90_lfpw_using_dew_point_depression,max_p95_lfpw_using_dew_point_depression,mean_p95_lfpw_using_dew_point_depression,median_p95_lfpw_using_dew_point_depression,p75_p95_lfpw_using_dew_point_depression,p90_p95_lfpw_using_dew_point_depression,p95_p95_lfpw_using_dew_point_depression,Date,Year
731,2017-01-01T00:00:00-08:00,HI LINE,0.208929,0.184574,0.177877,0.198159,0.206664,0.207288,0.136093,0.118948,...,0.742651,0.746579,1.584156,0.740645,0.711297,0.872378,1.088281,1.391995,2017-01-01,2017
732,2017-01-02T00:00:00-08:00,HI LINE,0.173715,0.127437,0.122416,0.130896,0.140195,0.141386,0.107618,0.079938,...,0.271703,0.295285,0.662477,0.240534,0.181829,0.268309,0.503919,0.504296,2017-01-02,2017
733,2017-01-03T00:00:00-08:00,HI LINE,0.115260,0.097902,0.094318,0.103164,0.107155,0.110092,0.082707,0.073263,...,0.339755,0.351245,0.512526,0.218543,0.223627,0.249231,0.370046,0.403352,2017-01-03,2017
734,2017-01-04T00:00:00-08:00,HI LINE,0.144602,0.124760,0.124227,0.138062,0.142129,0.144285,0.107508,0.091841,...,0.449087,0.577284,0.842947,0.262976,0.133749,0.510023,0.717174,0.837780,2017-01-04,2017
735,2017-01-05T00:00:00-08:00,HI LINE,0.105518,0.093871,0.091829,0.099657,0.104600,0.105109,0.078838,0.067325,...,0.079074,0.108244,0.336717,0.086051,0.063820,0.092996,0.148991,0.165740,2017-01-05,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14382138,2024-12-27T00:00:00-08:00,STROSNIDER,0.356850,0.306157,0.294632,0.319434,0.354444,0.356225,0.316661,0.266713,...,10.377188,11.190470,11.296192,4.341035,3.299156,5.012625,10.377188,11.190470,2024-12-27,2024
14382139,2024-12-28T00:00:00-08:00,STROSNIDER,0.318490,0.286921,0.281609,0.287351,0.315855,0.318211,0.284863,0.251467,...,5.885743,6.253679,6.687046,3.373338,3.179719,4.097660,5.885743,6.253679,2024-12-28,2024
14382140,2024-12-29T00:00:00-08:00,STROSNIDER,0.303039,0.277896,0.284506,0.290719,0.294680,0.295246,0.271280,0.247216,...,7.282709,8.666787,9.183432,4.572004,4.422780,5.253398,7.282709,8.666787,2024-12-29,2024
14382141,2024-12-30T00:00:00-08:00,STROSNIDER,0.342299,0.304461,0.297286,0.328365,0.339061,0.341265,0.316545,0.273253,...,2.859010,3.464509,3.847372,1.315146,0.887706,2.266536,2.859010,3.464509,2024-12-30,2024


In [ ]:
# read circuit lengths
circuitlengths = pd.read_excel("//sce.eix.com/workgroup/SPE_DSA/Charles/IgnitionGoalSettingInput/Q1_2025-_HFRA_Conductor_SumStat_Circuit.xlsx")
# filter for OH only
circuitlengths = circuitlengths[circuitlengths['Conductor Type'].str.contains('OH',na=False)]
# filter for HFRA only
circuitlengths = circuitlengths[circuitlengths['HFRA Tier'].notna()]
# remove blank circuits
circuitlengths = circuitlengths[circuitlengths['Circuit Name'].notna()]
# capitalize circuit names
circuitlengths['Circuit Name'] = circuitlengths['Circuit Name'].str.upper()
circuitlengths_map = circuitlengths.groupby(['Circuit Name','Circuit FLOC'])['Circuit Miles'].sum().reset_index()
# get Circuit FLOC 
circuitlengths_map

,Circuit Name,Circuit FLOC,Circuit Miles
0,ABACUS,ED-00005,1.592161
1,ABRAHAM,ED-00024,1.204734
2,ACADEMY,ED-00045,14.998395
3,ACADIAN,ED-00032,3.743688
4,ACAPULCO,ED-00041,5.780607
...,...,...,...
1614,ZEVO,ED-19826,0.061934
1615,ZINC,ED-19890,7.881998
1616,ZIRCON,ED-19835,0.519011
1617,ZONE,ED-19850,44.549072


In [ ]:
distribution_data

,time,id,max_max_fpi_2.0_tdd_fuels_index,mean_max_fpi_2.0_tdd_fuels_index,median_max_fpi_2.0_tdd_fuels_index,p75_max_fpi_2.0_tdd_fuels_index,p90_max_fpi_2.0_tdd_fuels_index,p95_max_fpi_2.0_tdd_fuels_index,max_mean_fpi_2.0_tdd_fuels_index,mean_mean_fpi_2.0_tdd_fuels_index,...,p90_p90_lfpw_using_dew_point_depression,p95_p90_lfpw_using_dew_point_depression,max_p95_lfpw_using_dew_point_depression,mean_p95_lfpw_using_dew_point_depression,median_p95_lfpw_using_dew_point_depression,p75_p95_lfpw_using_dew_point_depression,p90_p95_lfpw_using_dew_point_depression,p95_p95_lfpw_using_dew_point_depression,Date,Year
731,2017-01-01T00:00:00-08:00,HI LINE,0.208929,0.184574,0.177877,0.198159,0.206664,0.207288,0.136093,0.118948,...,0.742651,0.746579,1.584156,0.740645,0.711297,0.872378,1.088281,1.391995,2017-01-01,2017
732,2017-01-02T00:00:00-08:00,HI LINE,0.173715,0.127437,0.122416,0.130896,0.140195,0.141386,0.107618,0.079938,...,0.271703,0.295285,0.662477,0.240534,0.181829,0.268309,0.503919,0.504296,2017-01-02,2017
733,2017-01-03T00:00:00-08:00,HI LINE,0.115260,0.097902,0.094318,0.103164,0.107155,0.110092,0.082707,0.073263,...,0.339755,0.351245,0.512526,0.218543,0.223627,0.249231,0.370046,0.403352,2017-01-03,2017
734,2017-01-04T00:00:00-08:00,HI LINE,0.144602,0.124760,0.124227,0.138062,0.142129,0.144285,0.107508,0.091841,...,0.449087,0.577284,0.842947,0.262976,0.133749,0.510023,0.717174,0.837780,2017-01-04,2017
735,2017-01-05T00:00:00-08:00,HI LINE,0.105518,0.093871,0.091829,0.099657,0.104600,0.105109,0.078838,0.067325,...,0.079074,0.108244,0.336717,0.086051,0.063820,0.092996,0.148991,0.165740,2017-01-05,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14382138,2024-12-27T00:00:00-08:00,STROSNIDER,0.356850,0.306157,0.294632,0.319434,0.354444,0.356225,0.316661,0.266713,...,10.377188,11.190470,11.296192,4.341035,3.299156,5.012625,10.377188,11.190470,2024-12-27,2024
14382139,2024-12-28T00:00:00-08:00,STROSNIDER,0.318490,0.286921,0.281609,0.287351,0.315855,0.318211,0.284863,0.251467,...,5.885743,6.253679,6.687046,3.373338,3.179719,4.097660,5.885743,6.253679,2024-12-28,2024
14382140,2024-12-29T00:00:00-08:00,STROSNIDER,0.303039,0.277896,0.284506,0.290719,0.294680,0.295246,0.271280,0.247216,...,7.282709,8.666787,9.183432,4.572004,4.422780,5.253398,7.282709,8.666787,2024-12-29,2024
14382141,2024-12-30T00:00:00-08:00,STROSNIDER,0.342299,0.304461,0.297286,0.328365,0.339061,0.341265,0.316545,0.273253,...,2.859010,3.464509,3.847372,1.315146,0.887706,2.266536,2.859010,3.464509,2024-12-30,2024


In [ ]:
# read forecast distribution data
dist_df_forecast = pd.read_csv('//sce.eix.com/workgroup/SPE_DSA/Charles/IgnitionGoalSettingMidput/Dist_Forecasts_2025.csv', low_memory=False)
# make sure Date is date string of Datetime
dist_df_forecast['Date'] = pd.to_datetime(dist_df_forecast['Datetime']).dt.strftime('%Y-%m-%d')

In [ ]:
dist_df_forecast

,Datetime,Date,Circuit,FuelsIndex_max_FPI2,LFPwtdd_max_FPI2,Max_Wind,Max_Gust
0,2025-01-01 00:00:00,2025-01-01,ABACUS,0.45,1.28,6.51,7.40
1,2025-01-01 03:00:00,2025-01-01,ABACUS,0.45,1.52,6.75,7.72
2,2025-01-01 06:00:00,2025-01-01,ABACUS,0.42,0.10,1.66,1.73
3,2025-01-01 09:00:00,2025-01-01,ABACUS,0.48,0.21,2.37,2.49
4,2025-01-01 12:00:00,2025-01-01,ABACUS,0.51,0.49,3.18,4.55
...,...,...,...,...,...,...,...
12940958,2025-08-21 09:00:00,2025-08-21,ZONE,0.89,0.20,2.97,4.10
12940959,2025-08-21 12:00:00,2025-08-21,ZONE,0.88,2.78,8.72,15.83
12940960,2025-08-21 15:00:00,2025-08-21,ZONE,0.92,4.93,10.96,19.35
12940961,2025-08-21 18:00:00,2025-08-21,ZONE,0.92,3.94,10.20,18.00


In [ ]:
# for forecast data, groupby Date, Circuit, take the mean of FuelsIndex_max_FPI2
dist_df_forecast_agg = dist_df_forecast.groupby(['Date', 'Circuit'])['FuelsIndex_max_FPI2'].mean().reset_index()
# merge with circuit lengths
dist_df_forecast_agg = dist_df_forecast_agg.merge(circuitlengths_map, left_on='Circuit', right_on='Circuit Name', how='left')
# locate rows where Circuit Miles is NaN and Circuit contains ambercrest and change value to 32.7754002840092
dist_df_forecast_agg.loc[dist_df_forecast_agg['Circuit'].str.contains('AMBERCREST', na=False) & dist_df_forecast_agg['Circuit Miles'].isna(), 'Circuit Miles'] = 32.7754002840092
# locate rows where Circuit Miles is NaN and Circuit contains aventura and change value to 15.7032769637841
dist_df_forecast_agg.loc[dist_df_forecast_agg['Circuit'].str.contains('AVENTURA', na=False) & dist_df_forecast_agg['Circuit Miles'].isna(), 'Circuit Miles'] = 15.7032769637841
# locate rows where Circuit Miles is NaN and Circuit contains birchim and change value to 46.34484383
dist_df_forecast_agg.loc[dist_df_forecast_agg['Circuit'].str.contains('BIRCHIM', na=False) & dist_df_forecast_agg['Circuit Miles'].isna(), 'Circuit Miles'] = 46.34484383
# locate rows where Circuit Miles is NaN and Circuit contains cafe and change value to 27.6402669615555
dist_df_forecast_agg.loc[dist_df_forecast_agg['Circuit'].str.contains('CAFE', na=False) & dist_df_forecast_agg['Circuit Miles'].isna(), 'Circuit Miles'] = 27.6402669615555
# locate rows where Circuit Miles is NaN and Circuit contains canebrake and change value to 66.16859376
dist_df_forecast_agg.loc[dist_df_forecast_agg['Circuit'].str.contains('CANEBRAKE', na=False) & dist_df_forecast_agg['Circuit Miles'].isna(), 'Circuit Miles'] = 66.16859376
# locate rows where Circuit Miles is NaN and Circuit contains casals and change value to 0.446999553409736
dist_df_forecast_agg.loc[dist_df_forecast_agg['Circuit'].str.contains('CASALS', na=False) & dist_df_forecast_agg['Circuit Miles'].isna(), 'Circuit Miles'] = 0.446999553409736
# locate rows where Circuit Miles is NaN and Circuit contains coachella and change value to 6.590021052
dist_df_forecast_agg.loc[dist_df_forecast_agg['Circuit'].str.contains('COACHELLA', na=False) & dist_df_forecast_agg['Circuit Miles'].isna(), 'Circuit Miles'] = 6.590021052
# locate rows where circuit miles is NaN and circuit contains energy and change value to 32.37798311
dist_df_forecast_agg.loc[dist_df_forecast_agg['Circuit'].str.contains('ENERGY', na=False) & dist_df_forecast_agg['Circuit Miles'].isna(), 'Circuit Miles'] = 32.37798311
# locate rows where circuit miles is NaN and circuit contains fingal and change value to 36.6548594697613
dist_df_forecast_agg.loc[dist_df_forecast_agg['Circuit'].str.contains('FINGAL', na=False) & dist_df_forecast_agg['Circuit Miles'].isna(), 'Circuit Miles'] = 36.6548594697613
# locate rows where circuit miles is NaN and circuit contains guitar and change value to 42.2491638220717
dist_df_forecast_agg.loc[dist_df_forecast_agg['Circuit'].str.contains('GUITAR', na=False) & dist_df_forecast_agg['Circuit Miles'].isna(), 'Circuit Miles'] = 42.2491638220717
# locate rows where circuit miles is NaN and circuit contains huckleberry and change value to 25.4840533042463
dist_df_forecast_agg.loc[dist_df_forecast_agg['Circuit'].str.contains('HUCKLEBERRY', na=False) & dist_df_forecast_agg['Circuit Miles'].isna(), 'Circuit Miles'] = 25.4840533042463
dist_df_forecast_agg

,Date,Circuit,FuelsIndex_max_FPI2,Circuit Name,Circuit FLOC,Circuit Miles
0,2025-01-01,ABACUS,0.47875,ABACUS,ED-00005,1.592161
1,2025-01-01,ABRAHAM,0.66875,ABRAHAM,ED-00024,1.204734
2,2025-01-01,ACADEMY,0.65000,ACADEMY,ED-00045,14.998395
3,2025-01-01,ACADIAN,0.62500,ACADIAN,ED-00032,3.743688
4,2025-01-01,ACAPULCO,0.51500,ACAPULCO,ED-00041,5.780607
...,...,...,...,...,...,...
263971,2025-08-21,ZENDA,0.92625,ZENDA,ED-19820,48.447842
263972,2025-08-21,ZEVO,0.87625,ZEVO,ED-19826,0.061934
263973,2025-08-21,ZINC,0.91250,ZINC,ED-19890,7.881998
263974,2025-08-21,ZIRCON,0.91375,ZIRCON,ED-19835,0.519011


In [ ]:
dist_df_forecast_agg[dist_df_forecast_agg['Circuit Miles'].isna()]['Circuit'].unique()

array(['CITY OF BANNING #2', 'FROZEN', 'GRAPEVINE PEAK', 'HORSE MOUNTAIN',
       'INSPIRATION', 'JONAGOLD', 'KINSEY_01', 'KINSEY_02', 'KINSEY_03',
       'KINSEY_04', 'KINSEY_05', 'KINSEY_06', 'KINSEY_07', 'MAPLE',
       'PENSTOCK_01', 'PENSTOCK_02', 'PENSTOCK_03', 'PENSTOCK_04',
       'PENSTOCK_05', 'PENSTOCK_06', 'PENSTOCK_07', 'PENSTOCK_08',
       'PENSTOCK_09', 'PENSTOCK_10', 'PENSTOCK_11', 'POPPET FLATS_01',
       'POPPET FLATS_02', 'ROVER', 'SHOVEL_01', 'SHOVEL_02', 'SHOVEL_03',
       'SHOVEL_04', 'SILVERTONE', 'SUMMERHILL', 'SUTT_01', 'SUTT_02',
       'SUTT_03', 'SUTT_04', 'SUTT_05', 'SUTT_06', 'SUTT_07', 'TEJON_01',
       'TEJON_03', 'TEJON_04', 'TEJON_05', 'TEJON_06', 'TEJON_07',
       'TEJON_08', 'TOLL_01', 'TOLL_02', 'TUSSING', 'YANKEE', 'SHOVEL_05',
       'AMBUSHERS', 'CHIME', 'COLFAX', 'EVANS', 'FINISHLINE', 'GLACIER',
       'GOLDBUCKLE', 'HARLOW', 'HARWOOD', 'HOLLY VISTA', 'IDAHO',
       'LOLITA', 'MARLENE', 'MATERA', 'MERLOT', 'NORWOOD', 'PALA',
       'PHOEN

In [ ]:
# read ignition data
ignitions = pd.read_excel("//sce.eix.com/workgroup/SPE_DSA/Charles/IgnitionGoalSettingInput/Ignition_Safari_Connection.xlsx")
# rename columns 
ignitions = ignitions.rename(columns={'Date/Time':'Date','Circuit Name':'Circuit_Name','Circuit ID':'Circuit_ID','Event ID':'Event_ID'})
# get year column
ignitions['Year'] = pd.to_datetime(ignitions['Date']).dt.year
# keep only SCE involved ignitions
ignitions = ignitions[ignitions['SCE Involved'].str.contains('yes|inconclusive',case=False,na=False)]
# keep only reportable ignitions
ignitions = ignitions[ignitions['CPUC Reportable']==1]
# keep only HFRA ignitions
ignitions = ignitions[ignitions['HFRA'].str.contains('T2|T3',na=False)]
# loc row where circuit name has 'banducci' and change Circuit_ID to ET-02156
ignitions.loc[ignitions['Circuit_Name'].str.contains('banducci', case=False, na=False), 'Circuit_ID'] = 'ET-02156'
# change date format
ignitions['Date'] = pd.to_datetime(ignitions['Date']).dt.strftime('%Y-%m-%d')
# from 2017 onward
ignitions = ignitions[ignitions['Year'] >= 2017]
# # before August 1 2025
# ignitions = ignitions[ignitions['Date'] < '2025-08-01']
# group by Year and get count of ignitions
ignitions = ignitions.groupby('Year').agg({'Event_ID': 'count'}).reset_index()
ignitions

,Year,Event_ID
0,2017.0,34
1,2018.0,37
2,2019.0,38
3,2020.0,51
4,2021.0,48
5,2022.0,41
6,2023.0,19
7,2024.0,48
8,2025.0,24


Partitions that worked:

Fuel aggreggation: mean_max

Cutoff: 0.805

Weights for first, second, third years: [0.085, 0.18, 0.735]

Fuel Aggregation: max_max

Cutoff: 0.833

Weights for first, second, third years: [0.05, 0.1, 0.85]

Fuel Aggregation: mean_max

Cutoff: 0.8 (Make sure to address circuits with '_' in them, like Energy and Birchim)

Weights for first, second, third years: [0.02, 0.03, 0.95]

-Top Partitions Assumes 2024 had 46 ignitions, not 48

If 48 is the cutoff:

Fuel aggreggation: mean_max

Cutoff: 0.807

Weights: [0.04, 0.04, 0.92]

Fuel aggregation: mean_max

Cutoff: 0.81

Weights: [0.15, 0, 0.85]

In [225]:
dist_df_forecast_agg

,Date,Circuit,FuelsIndex_max_FPI2,Circuit Name,Circuit FLOC,Circuit Miles
0,2025-01-01,ABACUS,0.47875,ABACUS,ED-00005,1.592161
1,2025-01-01,ABRAHAM,0.66875,ABRAHAM,ED-00024,1.204734
2,2025-01-01,ACADEMY,0.65000,ACADEMY,ED-00045,14.998395
3,2025-01-01,ACADIAN,0.62500,ACADIAN,ED-00032,3.743688
4,2025-01-01,ACAPULCO,0.51500,ACAPULCO,ED-00041,5.780607
...,...,...,...,...,...,...
263971,2025-08-21,ZENDA,0.92625,ZENDA,ED-19820,48.447842
263972,2025-08-21,ZEVO,0.87625,ZEVO,ED-19826,0.061934
263973,2025-08-21,ZINC,0.91250,ZINC,ED-19890,7.881998
263974,2025-08-21,ZIRCON,0.91375,ZIRCON,ED-19835,0.519011


In [ ]:
# create a copy of distribution data
ads_agg = 'mean_max'
fuel_var = f"{ads_agg}_fpi_2.0_tdd_fuels_index"
fuel_var2 = 'FuelsIndex_max_FPI2'
distribution_data_cp = distribution_data[['Date','Year','id',fuel_var]].copy()
# read forecasted copy of distribution data
distribution_data_cp_forecast = dist_df_forecast_agg[['Date','Circuit Name',fuel_var2]].copy()
# rename id column to Circuit Name
distribution_data_cp.rename(columns={'id': 'Circuit Name'}, inplace=True)
# rename forecast data id column and fuel variable
distribution_data_cp_forecast.rename(columns={fuel_var2: fuel_var}, inplace=True)
# concat
distribution_data_cp = pd.concat([distribution_data_cp, distribution_data_cp_forecast], ignore_index=True)
# merge with circuitlengths_map
distribution_data_cp = distribution_data_cp.merge(circuitlengths_map, on='Circuit Name', how='left')
# drop na Circuit Miles
distribution_data_cp = distribution_data_cp[distribution_data_cp['Circuit Miles'].notna()]
# categorize fuel_var into above and below 0.8
distribution_data_cp['Fuel Category'] = np.where(distribution_data_cp[fuel_var] >= 0.8, 'High', 'Low')
# create year column and aggregate by year, then fuel category
distribution_data_cp['Year'] = pd.to_datetime(distribution_data_cp['Date']).dt.year
# save a temporary file for Dashboard to use
distribution_data_cp.to_csv("//sce.eix.com/workgroup/SPE_DSA/Charles/IgnitionGoalSettingOutput/CircuitData_Dashboard.csv",index=False)

In [ ]:
# determine fuel_var to use
ads_agg = 'mean_max'
fuel_var = f'{ads_agg}_fpi_2.0_tdd_fuels_index'
ads_agg2 = 'max_max'
wind_var = f'{ads_agg2}_lfpw_using_dew_point_depression'
# create a copy of the distribution data
res_df = distribution_data[['Date','Year','id',fuel_var, wind_var]].copy()
# rename id column to Circuit Name
res_df.rename(columns={'id': 'Circuit Name'}, inplace=True)
# create a new column called adjusted fuel_var
# use the rolling mean of the fuel_var over a 30-day window
res_df[f'{fuel_var}_rolling'] = res_df[fuel_var].rolling(window=14, min_periods=1).mean()
# merge with circuitlengths_map
res_df = res_df.merge(circuitlengths_map, on='Circuit Name', how='left')
# filter for only 2019-2024 data
res_df = res_df[res_df['Year'].between(2017, 2024)]
# drop the na Circuit Miles
res_df = res_df[res_df['Circuit Miles'].notna()]
# categorize fuel_var into above and below 0.7
fuel_val = 0.8
# create a second fuel_val for accounting wind
fuel_val2 = fuel_val
wind_val = 0
res_df['Fuel Category'] = np.where((res_df[fuel_var] >= fuel_val) | ((res_df[fuel_var] >= fuel_val2) & (res_df[wind_var] >= wind_val)), f'High', 'Low')
# # save a temporary file for Dashboard to use
# res_df.to_csv("CircuitLevelData/CircuitData_Dashboard.csv",index=False)
# set fuel category based on rolling mean
res_df['Fuel Category'] = np.where((res_df[f'{fuel_var}_rolling'] >= fuel_val) | ((res_df[f'{fuel_var}_rolling'] >= fuel_val2) & (res_df[wind_var] >= wind_val)), f'High', 'Low')
# aggregate by Year and Fuel Category
res_df = res_df.groupby(['Year', 'Fuel Category']).agg({'Circuit Miles': 'sum'}).reset_index()
# only keep the high fuel category
res_df = res_df[res_df['Fuel Category'] == 'High']
# merge with ignitions data
res_df = res_df.merge(ignitions, on='Year', how='left')
# locate where Year is 2019 and change Event_ID to 41
res_df.loc[res_df['Year'] == 2019, 'Event_ID'] = 41
# locate where Year is 2020 and change Event_ID to 51
res_df.loc[res_df['Year'] == 2020, 'Event_ID'] = 51
# locate where Year is 2024 and change Event_ID to 48
res_df.loc[res_df['Year'] == 2024, 'Event_ID'] = 48
# # locate where year is 2025 and change event_id to 21
# res_df.loc[res_df['Year'] == 2025, 'Event_ID'] = 21
# get the ratio by dividing Event_ID by Circuit Miles
res_df['Ignition Ratio'] = res_df['Event_ID'] / res_df['Circuit Miles']

# create another dataframe for the forecast data
res_df_forecast = dist_df_forecast_agg[['Date', 'Circuit', 'FuelsIndex_max_FPI2', 'Circuit Miles']].copy()
# create a year column
res_df_forecast['Year'] = pd.to_datetime(res_df_forecast['Date']).dt.year
# categorize fuel_var into above and below 0.7
res_df_forecast['Fuel Category'] = np.where((res_df_forecast['FuelsIndex_max_FPI2'] >= fuel_val), f'High', 'Low')
# aggregate by Year and Fuel Category
res_df_forecast = res_df_forecast.groupby(['Year', 'Fuel Category']).agg({'Circuit Miles': 'sum',}).reset_index()
# filter for High fuel category
res_df_forecast = res_df_forecast[res_df_forecast['Fuel Category'] == 'High']
# merge with ignitions data
res_df_forecast = res_df_forecast.merge(ignitions, on='Year', how='left')
# get the ratio by dividing Event_ID by Circuit Miles
res_df_forecast['Ignition Ratio'] = res_df_forecast['Event_ID'] / res_df_forecast['Circuit Miles']

# combine the two dataframes
res_df = pd.concat([res_df, res_df_forecast], ignore_index=True)
res_df

,Year,Fuel Category,Circuit Miles,Event_ID,Ignition Ratio
0,2017,High,1.402029e+06,34,0.000024
1,2018,High,1.540820e+06,37,0.000024
2,2019,High,1.391154e+06,41,0.000029
3,2020,High,1.498870e+06,51,0.000034
4,2021,High,1.769002e+06,48,0.000027
5,2022,High,1.459402e+06,41,0.000028
6,2023,High,5.846884e+05,19,0.000032
7,2024,High,1.434320e+06,48,0.000033
8,2025,High,7.533692e+05,24,0.000032


In [199]:
pd.Series(res_df.loc[res_df['Year'].isin([2020,2021,2022]),'Ignition Ratio']).std()*1.5

np.float64(5.599251644209923e-06)

In [200]:
### do a sorted version
table = []
for yr in range(2021, 2026):
    start, end = yr-3, yr-1
    # filter the data for the last 3 years
    filtered_data = res_df[(res_df['Year'] >= start) & (res_df['Year'] <= end)]
    # sort by Ignition Ratio in ascending order
    filtered_data = filtered_data.sort_values(by='Ignition Ratio')
    # assign weights to first, second and third year
    weights = [0.1, 0.2, 0.7]
    filtered_data['weights'] = weights
    # calculate the weighted average of Ignition Ratio
    weighted_avg = (filtered_data['Ignition Ratio'] * filtered_data['weights']).sum() / filtered_data['weights'].sum()
    target = weighted_avg
    print(target)
    # get the standard deviation of Ignition Ratio and multiply by 1.5 to get bound
    bound = filtered_data['Ignition Ratio'].std() * 1.5
    # get p80 by adding the bound to the ratio
    p80 = target + bound
    # get p20 by subtracting the bound from the ratio
    p20 = target - bound
    # get the actual ratio for the current year
    actual = res_df[res_df['Year'] == yr]['Ignition Ratio'].values
    # append the result to the table
    table.append({
        'Year': yr,
        'Actual': actual[0],
        'P20': p20, 
        'Target': target,
        'P80': p80,
        'Bound': bound
    })
# convert the table to a DataFrame
result_table = pd.DataFrame(table)
result_table

3.211364911968434e-05
3.242572442601894e-05
3.215007460000568e-05
3.107929299107733e-05
3.273430001306594e-05


,Year,Actual,P20,Target,P80,Bound
0,2021,0.000027,0.000025,0.000032,0.000040,0.000008
1,2022,0.000028,0.000027,0.000032,0.000038,0.000005
2,2023,0.000032,0.000027,0.000032,0.000038,0.000006
3,2024,0.000033,0.000027,0.000031,0.000035,0.000004
4,2025,0.000032,0.000028,0.000033,0.000037,0.000004


In [223]:
# create a result table
table = []
for yr in range(2021, 2027):
    start, end = yr-3, yr-1
    # filter the data for the last 3 years
    filtered_data = res_df[(res_df['Year'] >= start) & (res_df['Year'] <= end)]
    # assign weights to first, second and third year
    weights = [0.1, 0.2, 0.7]
    filtered_data['weights'] = weights
    # calculate the weighted average of Ignition Ratio
    weighted_avg = (filtered_data['Ignition Ratio'] * filtered_data['weights']).sum() / filtered_data['weights'].sum()
    target = weighted_avg
    # get the standard deviation of Ignition Ratio and multiply by 1.5 to get bound
    bound = filtered_data['Ignition Ratio'].std() * 1.5
    # get p80 by adding the bound to the ratio
    p80 = target + bound
    # get p20 by subtracting the bound from the ratio
    p20 = target - bound
    # get the actual ratio for the current year
    actual = res_df[res_df['Year'] == yr]['Ignition Ratio'].values
    if actual.size == 0:
        actual = [0]
    # append the result to the table
    table.append({
        'Year': yr,
        'Actual': actual[0],
        'P20': p20, 
        'Target': target,
        'P80': p80,
        'Bound': bound
    })
# convert the table to a DataFrame
result_table = pd.DataFrame(table)
result_table

C:\Users\tongcj\AppData\Local\Temp\ipykernel_24344\4046562280.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['weights'] = weights
C:\Users\tongcj\AppData\Local\Temp\ipykernel_24344\4046562280.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['weights'] = weights
C:\Users\tongcj\AppData\Local\Temp\ipykernel_24344\4046562280.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

,Year,Actual,P20,Target,P80,Bound
0,2021,0.000027,0.000025,0.000032,0.000040,0.000008
1,2022,0.000028,0.000023,0.000029,0.000034,0.000005
2,2023,0.000032,0.000023,0.000028,0.000034,0.000006
3,2024,0.000033,0.000027,0.000031,0.000035,0.000004
4,2025,0.000032,0.000028,0.000033,0.000037,0.000004
5,2026,0.000000,0.000031,0.000032,0.000033,0.000001


In [ ]:
# save res_df and result_table to an excel file
res_df.to_excel(f"//sce.eix.com/workgroup/SPE_DSA/Charles/IgnitionGoalSettingOutput/Circuit_Level_Ignition_Rate_Dashboard.xlsx", index=False)
# add a sheet to the excel file with the result table
with pd.ExcelWriter(f"//sce.eix.com/workgroup/SPE_DSA/Charles/IgnitionGoalSettingOutput/Circuit_Level_Ignition_Rate_Dashboard.xlsx", engine='openpyxl', mode='a') as writer:
    result_table.to_excel(writer, sheet_name='Result Table', index=False)

In [ ]:
# get month over month performance for circuit
ignitions_2025 = pd.read_excel("//sce.eix.com/workgroup/SPE_DSA/Charles/IgnitionGoalSettingInput/Ignition_Safari_Connection.xlsx")
# rename columns 
ignitions_2025 = ignitions_2025.rename(columns={'Date/Time':'Date','Circuit Name':'Circuit_Name','Circuit ID':'Circuit_ID','Event ID':'Event_ID'})
# get year column
ignitions_2025['Year'] = pd.to_datetime(ignitions_2025['Date']).dt.year
# keep only 2025 in Year
ignitions_2025 = ignitions_2025[ignitions_2025['Year'] == 2025]
# get month column
ignitions_2025['Month'] = pd.to_datetime(ignitions_2025['Date']).dt.month
# keep only SCE involved ignitions
ignitions_2025 = ignitions_2025[ignitions_2025['SCE Involved'].str.contains('yes|inconclusive',case=False,na=False)]
# keep only reportable ignitions
ignitions_2025 = ignitions_2025[ignitions_2025['CPUC Reportable']==1]
# keep only HFRA ignitions
ignitions_2025 = ignitions_2025[ignitions_2025['HFRA'].str.contains('T2|T3',na=False)]
# loc row where circuit name has 'banducci' and change Circuit_ID to ET-02156
ignitions_2025.loc[ignitions_2025['Circuit_Name'].str.contains('banducci', case=False, na=False), 'Circuit_ID'] = 'ET-02156'
# group by month
ignitions_2025_grouped = ignitions_2025.groupby(['Month']).size().reset_index(name='Counts')
# create a cumsum
ignitions_2025_grouped['CumsumCounts'] = ignitions_2025_grouped['Counts'].cumsum()

# create a copy of the aggreagated forecast file
dist_df_forecast_agg_cp = dist_df_forecast_agg.copy()
# create a month column
dist_df_forecast_agg_cp['Month'] = pd.to_datetime(dist_df_forecast_agg_cp['Date']).dt.month
# categorize fuel column
dist_df_forecast_agg_cp['Fuel_Category'] = np.where(dist_df_forecast_agg_cp['FuelsIndex_max_FPI2'] >= fuel_val, 'High', 'Low')
# group by fuel category
dist_df_forecast_agg_cp = dist_df_forecast_agg_cp.groupby(['Month','Fuel_Category']).agg({'Circuit Miles':'sum'}).reset_index()
# keep only High Category
dist_df_forecast_agg_cp = dist_df_forecast_agg_cp[dist_df_forecast_agg_cp['Fuel_Category'] == 'High']

# merge
ignitions_2025_grouped = ignitions_2025_grouped.merge(dist_df_forecast_agg_cp, on='Month', how='left')
# for miles, fillna with 0
ignitions_2025_grouped['Circuit Miles'] = ignitions_2025_grouped['Circuit Miles'].fillna(0)
# create a cumsum miles column
ignitions_2025_grouped['CumsumMiles'] = ignitions_2025_grouped['Circuit Miles'].cumsum()
# calculate cum sum rate
ignitions_2025_grouped['CumsumRate'] = ignitions_2025_grouped['CumsumCounts'] / ignitions_2025_grouped['CumsumMiles']
ignitions_2025_grouped

,Month,Counts,CumsumCounts,Fuel_Category,Circuit Miles,CumsumMiles,CumsumRate
0,1,4,4,High,50732.538959,50732.538959,0.000079
1,2,2,6,NaN,0.000000,50732.538959,0.000118
2,3,1,7,NaN,0.000000,50732.538959,0.000138
3,4,1,8,High,52.837532,50785.376491,0.000158
4,5,3,11,High,2744.930658,53530.307149,0.000205
5,6,6,17,High,141162.722756,194693.029905,0.000087
6,7,3,20,High,302426.071063,497119.100969,0.000040
7,8,4,24,High,256250.145963,753369.246932,0.000032


In [203]:
# # save res_df and result table to an excel file
# res_df.to_excel(f"Circuit_Level_Analysis_Distribution_Only_{fuel_val}.xlsx", index=False)
# # add a sheet to the excel file with the result table
# with pd.ExcelWriter(f"Circuit_Level_Analysis_Distribution_Only_{fuel_val}.xlsx", engine='openpyxl', mode='a') as writer:
#     result_table.to_excel(writer, sheet_name='Result Table', index=False)

In [204]:
# # create a result table
# table = []
# for yr in range(2022, 2025):
#     start, end = yr-3, yr-1
#     # filter the data for the last 3 years
#     filtered_data = res_df[(res_df['Year'] >= start) & (res_df['Year'] <= end)]
#     # get the ratio for the last 3 years to obtain target
#     target = filtered_data['Ignition Ratio'].mean()
#     # get the standard deviation of Ignition Ratio and multiply by 1.5 to get bound
#     bound = filtered_data['Ignition Ratio'].std() * 1.5
#     # get p80 by adding the bound to the ratio
#     p80 = target + bound
#     # get p20 by subtracting the bound from the ratio
#     p20 = target - bound
#     # get the actual ratio for the current year
#     actual = res_df[res_df['Year'] == yr]['Ignition Ratio'].values
#     # append the result to the table
#     table.append({
#         'Year': yr,
#         'Actual': actual[0],
#         'P20': p20, 
#         'Target': target,
#         'P80': p80,
#         'Bound': bound
#     })
# # convert the table to a DataFrame
# result_table = pd.DataFrame(table)
# result_table

In [205]:
# read distribution data
distribution_data

,time,id,max_max_fpi_2.0_tdd_fuels_index,mean_max_fpi_2.0_tdd_fuels_index,median_max_fpi_2.0_tdd_fuels_index,p75_max_fpi_2.0_tdd_fuels_index,p90_max_fpi_2.0_tdd_fuels_index,p95_max_fpi_2.0_tdd_fuels_index,max_mean_fpi_2.0_tdd_fuels_index,mean_mean_fpi_2.0_tdd_fuels_index,...,p90_p90_lfpw_using_dew_point_depression,p95_p90_lfpw_using_dew_point_depression,max_p95_lfpw_using_dew_point_depression,mean_p95_lfpw_using_dew_point_depression,median_p95_lfpw_using_dew_point_depression,p75_p95_lfpw_using_dew_point_depression,p90_p95_lfpw_using_dew_point_depression,p95_p95_lfpw_using_dew_point_depression,Date,Year
731,2017-01-01T00:00:00-08:00,HI LINE,0.208929,0.184574,0.177877,0.198159,0.206664,0.207288,0.136093,0.118948,...,0.742651,0.746579,1.584156,0.740645,0.711297,0.872378,1.088281,1.391995,2017-01-01,2017
732,2017-01-02T00:00:00-08:00,HI LINE,0.173715,0.127437,0.122416,0.130896,0.140195,0.141386,0.107618,0.079938,...,0.271703,0.295285,0.662477,0.240534,0.181829,0.268309,0.503919,0.504296,2017-01-02,2017
733,2017-01-03T00:00:00-08:00,HI LINE,0.115260,0.097902,0.094318,0.103164,0.107155,0.110092,0.082707,0.073263,...,0.339755,0.351245,0.512526,0.218543,0.223627,0.249231,0.370046,0.403352,2017-01-03,2017
734,2017-01-04T00:00:00-08:00,HI LINE,0.144602,0.124760,0.124227,0.138062,0.142129,0.144285,0.107508,0.091841,...,0.449087,0.577284,0.842947,0.262976,0.133749,0.510023,0.717174,0.837780,2017-01-04,2017
735,2017-01-05T00:00:00-08:00,HI LINE,0.105518,0.093871,0.091829,0.099657,0.104600,0.105109,0.078838,0.067325,...,0.079074,0.108244,0.336717,0.086051,0.063820,0.092996,0.148991,0.165740,2017-01-05,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14382138,2024-12-27T00:00:00-08:00,STROSNIDER,0.356850,0.306157,0.294632,0.319434,0.354444,0.356225,0.316661,0.266713,...,10.377188,11.190470,11.296192,4.341035,3.299156,5.012625,10.377188,11.190470,2024-12-27,2024
14382139,2024-12-28T00:00:00-08:00,STROSNIDER,0.318490,0.286921,0.281609,0.287351,0.315855,0.318211,0.284863,0.251467,...,5.885743,6.253679,6.687046,3.373338,3.179719,4.097660,5.885743,6.253679,2024-12-28,2024
14382140,2024-12-29T00:00:00-08:00,STROSNIDER,0.303039,0.277896,0.284506,0.290719,0.294680,0.295246,0.271280,0.247216,...,7.282709,8.666787,9.183432,4.572004,4.422780,5.253398,7.282709,8.666787,2024-12-29,2024
14382141,2024-12-30T00:00:00-08:00,STROSNIDER,0.342299,0.304461,0.297286,0.328365,0.339061,0.341265,0.316545,0.273253,...,2.859010,3.464509,3.847372,1.315146,0.887706,2.266536,2.859010,3.464509,2024-12-30,2024


## Coming up with Range for Aug-Dec

In [206]:
# get fuel data for july 2024 and july 2025
july_2024 = distribution_data[(distribution_data['Date'] >= '2024-07-01') & (distribution_data['Date'] < '2024-08-01')]
july_2025 = dist_df_forecast[(dist_df_forecast['Date'] >= '2025-07-01') & (dist_df_forecast['Date'] < '2025-08-01')]
# rename id to Circuit Name
july_2024.rename(columns={'id': 'Circuit Name'}, inplace=True)
july_2025.rename(columns={'Circuit': 'Circuit Name','FuelsIndex_max_FPI2':'max_max_fpi_2.0_tdd_fuels_index'}, inplace=True)

C:\Users\tongcj\AppData\Local\Temp\ipykernel_24344\1659929170.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  july_2024.rename(columns={'id': 'Circuit Name'}, inplace=True)
C:\Users\tongcj\AppData\Local\Temp\ipykernel_24344\1659929170.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  july_2025.rename(columns={'Circuit': 'Circuit Name','FuelsIndex_max_FPI2':'max_max_fpi_2.0_tdd_fuels_index'}, inplace=True)


In [207]:
july_2025

,Datetime,Date,Circuit Name,max_max_fpi_2.0_tdd_fuels_index,LFPwtdd_max_FPI2,Max_Wind,Max_Gust
11153717,2025-07-01 00:00:00,2025-07-01,ABACUS,0.90,0.34,3.06,4.05
11153774,2025-07-01 00:00:00,2025-07-01,ABRAHAM,0.85,0.10,2.91,3.78
11153831,2025-07-01 00:00:00,2025-07-01,ACADEMY,0.83,0.09,2.67,2.83
11153888,2025-07-01 00:00:00,2025-07-01,ACADIAN,0.89,0.30,3.40,3.65
11153945,2025-07-01 00:00:00,2025-07-01,ACAPULCO,0.79,0.26,4.58,5.82
...,...,...,...,...,...,...,...
12940790,2025-07-31 09:00:00,2025-07-31,ZONE,0.80,0.19,3.92,5.67
12940791,2025-07-31 12:00:00,2025-07-31,ZONE,0.85,2.11,10.06,17.96
12940792,2025-07-31 15:00:00,2025-07-31,ZONE,0.87,2.95,10.70,18.99
12940793,2025-07-31 18:00:00,2025-07-31,ZONE,0.88,2.85,10.41,18.40


In [208]:
july_2025[july_2025['Date']=='2025-07-07']

,Datetime,Date,Circuit Name,max_max_fpi_2.0_tdd_fuels_index,LFPwtdd_max_FPI2,Max_Wind,Max_Gust
11548043,2025-07-07 00:00:00,2025-07-07,ABACUS,0.88,0.16,2.14,2.55
11548100,2025-07-07 00:00:00,2025-07-07,ABRAHAM,0.87,0.02,1.24,1.33
11548157,2025-07-07 00:00:00,2025-07-07,ACADEMY,0.83,0.02,1.83,2.20
11548214,2025-07-07 00:00:00,2025-07-07,ACADIAN,0.89,0.59,4.28,4.66
11548271,2025-07-07 00:00:00,2025-07-07,ACAPULCO,0.77,0.15,4.96,5.47
...,...,...,...,...,...,...,...
12073701,2025-07-07 09:00:00,2025-07-07,ZONE,0.77,0.16,4.60,6.92
12073702,2025-07-07 12:00:00,2025-07-07,ZONE,0.82,1.68,9.78,17.58
12073703,2025-07-07 15:00:00,2025-07-07,ZONE,0.83,2.73,11.40,20.19
12073704,2025-07-07 18:00:00,2025-07-07,ZONE,0.85,2.10,10.38,18.41


In [209]:
# group 2024 data by Circuit Name and Date, take the max of FuelsIndex_max_FPI2
july_2024_agg = july_2024.groupby(['Circuit Name', 'Date'])['max_max_fpi_2.0_tdd_fuels_index'].max().reset_index()
# merge with circuit lengths
july_2024_agg = july_2024_agg.merge(circuitlengths_map, left_on='Circuit Name', right_on='Circuit Name', how='left')
# group 2025 data by Circuit and Date, take the max of FuelsIndex_max_FPI2
july_2025_agg = july_2025.groupby(['Circuit Name', 'Date'])['max_max_fpi_2.0_tdd_fuels_index'].max().reset_index()
# merge with circuit lengths
july_2025_agg = july_2025_agg.merge(circuitlengths_map, left_on='Circuit Name', right_on='Circuit Name', how='left')
# make sure 2024 circuits are in 2025 circuits data
july_2024_agg = july_2024_agg[july_2024_agg['Circuit Name'].isin(july_2025_agg['Circuit Name'])]
# categorize fuel data based on cutoff value
high_cutoff = 0.9
med_cutoff = 0.6
# create conditions and choices for categorization
def categorize_fuel(value):
    if value >= high_cutoff:
        return 'High'
    elif value >= med_cutoff:
        return 'Medium'
    else:
        return 'Low'
july_2024_agg['Fuel Category'] = july_2024_agg['max_max_fpi_2.0_tdd_fuels_index'].apply(categorize_fuel)
july_2025_agg['Fuel Category'] = july_2025_agg['max_max_fpi_2.0_tdd_fuels_index'].apply(categorize_fuel)
# group by Fuel Category and get the sum of Circuit Miles
july_2024_agg = july_2024_agg.groupby('Fuel Category').agg({'Circuit Miles': 'sum'}).reset_index()
july_2025_agg = july_2025_agg.groupby('Fuel Category').agg({'Circuit Miles': 'sum'}).reset_index()
# get the percentage of each Fuel Category
july_2024_agg['Percentage'] = july_2024_agg['Circuit Miles'] / july_2024_agg['Circuit Miles'].sum() * 100
july_2025_agg['Percentage'] = july_2025_agg['Circuit Miles'] / july_2025_agg['Circuit Miles'].sum() * 100
july_2024_agg

,Fuel Category,Circuit Miles,Percentage
0,High,180667.599179,51.096481
1,Low,3718.048476,1.051540
2,Medium,169195.649085,47.851979


In [210]:
july_2025_agg

,Fuel Category,Circuit Miles,Percentage
0,High,166736.717596,47.026859
1,Low,1714.126512,0.483457
2,Medium,186105.514915,52.489685


In [211]:
# get 2024 aggregation for each month from aug-dec
aug_2024 = distribution_data[(distribution_data['Date'] >= '2024-08-01') & (distribution_data['Date'] < '2024-09-01')]
sep_2024 = distribution_data[(distribution_data['Date'] >= '2024-09-01') & (distribution_data['Date'] < '2024-10-01')]
oct_2024 = distribution_data[(distribution_data['Date'] >= '2024-10-01') & (distribution_data['Date'] < '2024-11-01')]
nov_2024 = distribution_data[(distribution_data['Date'] >= '2024-11-01') & (distribution_data['Date'] < '2024-12-01')]
dec_2024 = distribution_data[(distribution_data['Date'] >= '2024-12-01') & (distribution_data['Date'] < '2025-01-01')]
# group by Circuit Name and Date, take the max of FuelsIndex_max_FPI2
aug_2024_agg = aug_2024.groupby(['id', 'Date'])['max_max_fpi_2.0_tdd_fuels_index'].max().reset_index()
sep_2024_agg = sep_2024.groupby(['id', 'Date'])['max_max_fpi_2.0_tdd_fuels_index'].max().reset_index()
oct_2024_agg = oct_2024.groupby(['id', 'Date'])['max_max_fpi_2.0_tdd_fuels_index'].max().reset_index()
nov_2024_agg = nov_2024.groupby(['id', 'Date'])['max_max_fpi_2.0_tdd_fuels_index'].max().reset_index()
dec_2024_agg = dec_2024.groupby(['id', 'Date'])['max_max_fpi_2.0_tdd_fuels_index'].max().reset_index()
# merge with circuit lengths
aug_2024_agg = aug_2024_agg.merge(circuitlengths_map, left_on='id', right_on='Circuit Name', how='left')
sep_2024_agg = sep_2024_agg.merge(circuitlengths_map, left_on='id', right_on='Circuit Name', how='left')
oct_2024_agg = oct_2024_agg.merge(circuitlengths_map, left_on='id', right_on='Circuit Name', how='left')
nov_2024_agg = nov_2024_agg.merge(circuitlengths_map, left_on='id', right_on='Circuit Name', how='left')
dec_2024_agg = dec_2024_agg.merge(circuitlengths_map, left_on='id', right_on='Circuit Name', how='left')
# make sure 2024 circuits are in 2025 circuits data
aug_2024_agg = aug_2024_agg[aug_2024_agg['id'].isin(july_2025['Circuit Name'])]
sep_2024_agg = sep_2024_agg[sep_2024_agg['id'].isin(july_2025['Circuit Name'])]
oct_2024_agg = oct_2024_agg[oct_2024_agg['id'].isin(july_2025['Circuit Name'])]
nov_2024_agg = nov_2024_agg[nov_2024_agg['id'].isin(july_2025['Circuit Name'])]
dec_2024_agg = dec_2024_agg[dec_2024_agg['id'].isin(july_2025['Circuit Name'])]
# categorize fuel data based on cutoff value
aug_2024_agg['Fuel Category'] = aug_2024_agg['max_max_fpi_2.0_tdd_fuels_index'].apply(categorize_fuel)
sep_2024_agg['Fuel Category'] = sep_2024_agg['max_max_fpi_2.0_tdd_fuels_index'].apply(categorize_fuel)
oct_2024_agg['Fuel Category'] = oct_2024_agg['max_max_fpi_2.0_tdd_fuels_index'].apply(categorize_fuel)
nov_2024_agg['Fuel Category'] = nov_2024_agg['max_max_fpi_2.0_tdd_fuels_index'].apply(categorize_fuel)
dec_2024_agg['Fuel Category'] = dec_2024_agg['max_max_fpi_2.0_tdd_fuels_index'].apply(categorize_fuel)
# group by Fuel Category and get the sum of Circuit Miles
aug_2024_agg = aug_2024_agg.groupby('Fuel Category').agg({'Circuit Miles': 'sum'}).reset_index()
sep_2024_agg = sep_2024_agg.groupby('Fuel Category').agg({'Circuit Miles': 'sum'}).reset_index()
oct_2024_agg = oct_2024_agg.groupby('Fuel Category').agg({'Circuit Miles': 'sum'}).reset_index()
nov_2024_agg = nov_2024_agg.groupby('Fuel Category').agg({'Circuit Miles': 'sum'}).reset_index()
dec_2024_agg = dec_2024_agg.groupby('Fuel Category').agg({'Circuit Miles': 'sum'}).reset_index()
# get the percentage of each Fuel Category
aug_2024_agg['Percentage'] = aug_2024_agg['Circuit Miles'] / aug_2024_agg['Circuit Miles'].sum() * 100
sep_2024_agg['Percentage'] = sep_2024_agg['Circuit Miles'] / sep_2024_agg['Circuit Miles'].sum() * 100
oct_2024_agg['Percentage'] = oct_2024_agg['Circuit Miles'] / oct_2024_agg['Circuit Miles'].sum() * 100
nov_2024_agg['Percentage'] = nov_2024_agg['Circuit Miles'] / nov_2024_agg['Circuit Miles'].sum() * 100
dec_2024_agg['Percentage'] = dec_2024_agg['Circuit Miles'] / dec_2024_agg['Circuit Miles'].sum() * 100

In [212]:
aug_2024_agg

,Fuel Category,Circuit Miles,Percentage
0,High,239949.814661,67.862700
1,Low,704.684515,0.199299
2,Medium,112926.797564,31.938001


In [213]:
sep_2024_agg

,Fuel Category,Circuit Miles,Percentage
0,High,184892.923556,54.034538
1,Low,806.203797,0.235611
2,Medium,156476.321105,45.729851


In [214]:
oct_2024_agg

,Fuel Category,Circuit Miles,Percentage
0,High,214907.249077,60.780152
1,Low,2440.565875,0.690242
2,Medium,136233.481788,38.529606


In [215]:
nov_2024_agg

,Fuel Category,Circuit Miles,Percentage
0,High,12519.954553,3.658928
1,Low,67757.958985,19.802110
2,Medium,261897.534921,76.538962


In [216]:
dec_2024_agg

,Fuel Category,Circuit Miles,Percentage
0,High,9385.329681,2.654363
1,Low,79258.621169,22.415954
2,Medium,264937.345890,74.929683


## Read Joey's NetCDF File of Precipitation

In [217]:
import xarray  as xr
import netCDF4
import h5netcdf

# load netcdf file
precip_2024 = xr.open_dataset(r"\\sce.eix.com\workgroup\SPE_DSA\DataMarts\ADS\Daily 7 Features - 4CAVA\ADS_daily_2024.nc")

# convert to dataframe
precip_2024 = precip_2024.to_dataframe().reset_index()

ValueError: found the following matches with the input file in xarray's IO backends: ['netcdf4', 'h5netcdf']. But their dependencies may not be installed, see:
https://docs.xarray.dev/en/stable/user-guide/io.html 
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html